In [ ]:
!pip install sklearn-crfsuite


In [ ]:
!wget https://www.cnts.ua.ac.be/conll2002/ner.tgz
!tar -xvf ner.tgz
!ls


In [ ]:
import gzip
import shutil

files = ["esp.train", "esp.testa", "esp.testb"]

for f in files:
    with gzip.open(f"ner/data/{f}.gz", 'rb') as fr, open(f"{f}", 'wb') as fw:
        shutil.copyfileobj(fr, fw)

!ls


esp.testa  esp.testb  esp.train  ner  ner.tgz  sample_data


In [ ]:
!head -n 30 esp.train


Melbourne B-LOC
( O
Australia B-LOC
) O
, O
25 O
may O
( O
EFE B-ORG
) O
. O

- O

El O
Abogado B-PER
General I-PER
del I-PER
Estado I-PER
, O
Daryl B-PER
Williams I-PER
, O
subray� O
hoy O
la O
necesidad O
de O
tomar O
medidas O


In [ ]:
def read_conll(path):
    sentences = []
    labels = []
    sent_tokens = []
    sent_labels = []

    with open(path, encoding="latin-1") as f:
        for line in f:
            line = line.strip()
            if not line:
                if sent_tokens:
                    sentences.append(sent_tokens)
                    labels.append(sent_labels)
                    sent_tokens, sent_labels = [], []
                continue

            parts = line.split()
            if len(parts) == 2:
                word, ner = parts
            else:
                # Bazen 3 sütun gelebiliyor
                word, ner = parts[0], parts[-1]
            sent_tokens.append(word)
            sent_labels.append(ner)

    if sent_tokens:
        sentences.append(sent_tokens)
        labels.append(sent_labels)

    return sentences, labels


In [ ]:
X_train_raw, y_train = read_conll("esp.train")
X_dev_raw,   y_dev   = read_conll("esp.testa")
X_test_raw,  y_test  = read_conll("esp.testb")


In [ ]:
print(X_train_raw[0][:10])
print(y_train[0][:10])


['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')']
['B-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O']


In [ ]:
import re

def word2features(sent, i):
    word = sent[i]
    wlow = word.lower()

    features = {
        "bias": 1.0,
        "word.lower": wlow,
        "word.isupper": word.isupper(),
        "word.istitle": word.istitle(),
        "word.isdigit": word.isdigit(),
        "suffix3": wlow[-3:],
        "prefix3": wlow[:3],
        "contains_dot": "." in word,
        "contains_dash": "-" in word,
    }

    if i > 0:
        prev = sent[i-1]
        features.update({
            "-1:lower": prev.lower(),
            "-1:istitle": prev.istitle(),
            "-1:isupper": prev.isupper(),
        })
    else:
        features["BOS"] = True  # cümle başı

    if i < len(sent)-1:
        nxt = sent[i+1]
        features.update({
            "+1:lower": nxt.lower(),
            "+1:istitle": nxt.istitle(),
            "+1:isupper": nxt.isupper(),
        })
    else:
        features["EOS"] = True  # cümle sonu

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(labels):
    return labels


In [ ]:
X_train = [sent2features(s) for s in X_train_raw]
X_dev   = [sent2features(s) for s in X_dev_raw]
X_test  = [sent2features(s) for s in X_test_raw]

y_train_crf = [sent2labels(s) for s in y_train]
y_dev_crf   = [sent2labels(s) for s in y_dev]
y_test_crf  = [sent2labels(s) for s in y_test]


In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics

crf = sklearn_crfsuite.CRF(
    algorithm="lbfgs",
    c1=0.05,
    c2=0.05,
    max_iterations=100,
    all_possible_transitions=True
)

crf.fit(X_train, y_train_crf)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.05, c2=0.05,
    max_iterations=100)

In [ ]:
y_pred = crf.predict(X_test)

print(metrics.flat_classification_report(
    y_test_crf, y_pred, digits=4
))


              precision    recall  f1-score   support

       B-LOC     0.8033    0.7685    0.7855      1084
      B-MISC     0.6846    0.5251    0.5943       339
       B-ORG     0.8131    0.8421    0.8274      1400
       B-PER     0.8507    0.8680    0.8593       735
       I-LOC     0.7076    0.6031    0.6512       325
      I-MISC     0.6092    0.5709    0.5894       557
       I-ORG     0.8365    0.7880    0.8116      1104
       I-PER     0.9088    0.9432    0.9257       634
           O     0.9918    0.9959    0.9938     45355

    accuracy                         0.9698     51533
   macro avg     0.8006    0.7672    0.7820     51533
weighted avg     0.9687    0.9698    0.9691     51533



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import scipy.stats

params_space = {
    "c1": scipy.stats.expon(scale=0.5),
    "c2": scipy.stats.expon(scale=0.05),
}

rs = RandomizedSearchCV(
    crf,
    params_space,
    cv=3,
    verbose=1,
    n_jobs=-1,
    scoring="f1_macro"
)

rs.fit(X_train, y_train_crf)

print("Best params:", rs.best_params_)
best_crf = rs.best_estimator_


In [ ]:
y_pred_boost = best_crf.predict(X_test)

print(metrics.flat_classification_report(
    y_test_crf,
    y_pred_boost,
    digits=4
))


              precision    recall  f1-score   support

       B-LOC     0.7874    0.7518    0.7692      1084
      B-MISC     0.6958    0.4926    0.5769       339
       B-ORG     0.8079    0.8143    0.8111      1400
       B-PER     0.8238    0.8585    0.8408       735
       I-LOC     0.5884    0.5938    0.5911       325
      I-MISC     0.5688    0.5566    0.5626       557
       I-ORG     0.8251    0.7609    0.7917      1104
       I-PER     0.8864    0.9353    0.9102       634
           O     0.9914    0.9950    0.9932     45355

    accuracy                         0.9667     51533
   macro avg     0.7750    0.7510    0.7608     51533
weighted avg     0.9658    0.9667    0.9661     51533



In [ ]:

# 🇪🇸 1) Mini lexicon'ları tanımla (LOC / DATE / ORG ipuçları)
SPANISH_COUNTRIES = {
    "españa","argentina","bolivia","chile","colombia","cuba","ecuador","méxico","uruguay",
    "venezuela","paraguay","perú","brasil","portugal","francia","alemania","italia"
}

SPANISH_MONTHS = {
    "enero","febrero","marzo","abril","mayo","junio","julio","agosto",
    "septiembre","octubre","noviembre","diciembre"
}

ORG_SUFFIXES = {
    "s.a.","s.l.","corp.","inc.","co.","grupo","bank","telecom","universidad"
}

In [ ]:
def word2features(sent, i):
    word = sent[i]
    wlow = word.lower()

    features = {
        "bias": 1.0,
        "word.lower": wlow,
        "word.isupper": word.isupper(),
        "word.istitle": word.istitle(),
        "word.isdigit": word.isdigit(),

        "prefix1": wlow[:1],
        "prefix2": wlow[:2],
        "prefix3": wlow[:3],
        "suffix1": wlow[-1:],
        "suffix2": wlow[-2:],
        "suffix3": wlow[-3:],

        "contains_dot": "." in word,
        "contains_dash": "-" in word,
    }

    # 🎁 LEXICON BOOST İPUÇLARI
    features["is_country"] = wlow in SPANISH_COUNTRIES
    features["is_month"] = wlow in SPANISH_MONTHS
    features["org_suffix"] = any(wlow.endswith(s) for s in ORG_SUFFIXES)

    # Bağlam özellikleri (context)
    if i > 0:
        prev = sent[i-1].lower()
        features.update({
            "-1.lower": prev,
            "-1.istitle": prev.istitle(),
            "-1.isupper": prev.isupper(),
        })
    else:
        features["BOS"] = True

    if i < len(sent) - 1:
        nxt = sent[i+1].lower()
        features.update({
            "+1.lower": nxt,
            "+1.istitle": nxt.istitle(),
            "+1.isupper": nxt.isupper(),
        })
    else:
        features["EOS"] = True

    return features


In [ ]:
X_train = [sent2features(s) for s in X_train_raw]
X_dev   = [sent2features(s) for s in X_dev_raw]
X_test  = [sent2features(s) for s in X_test_raw]

# 4️⃣ CRF modelini lexicon ile tekrar eğit
best_crf.fit(X_train, y_train_crf)

# 5️⃣ Test setinde tahmin yap ve rapor al
y_pred_lex = best_crf.predict(X_test)

from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(y_test_crf, y_pred_lex, digits=4))

              precision    recall  f1-score   support

       B-LOC     0.7988    0.7509    0.7741      1084
      B-MISC     0.6708    0.4808    0.5601       339
       B-ORG     0.8007    0.8236    0.8120      1400
       B-PER     0.8280    0.8776    0.8520       735
       I-LOC     0.6476    0.6277    0.6375       325
      I-MISC     0.6245    0.5583    0.5896       557
       I-ORG     0.8083    0.7409    0.7732      1104
       I-PER     0.8561    0.9385    0.8954       634
           O     0.9917    0.9956    0.9937     45355

    accuracy                         0.9675     51533
   macro avg     0.7807    0.7549    0.7653     51533
weighted avg     0.9663    0.9675    0.9667     51533



In [ ]:
labels_no_misc = [l for l in crf.classes_ if "MISC" not in l and l != "O"]

print(metrics.flat_classification_report(
    y_test_crf,
    y_pred_lex,
    labels=labels_no_misc,
    digits=4
))


              precision    recall  f1-score   support

       B-LOC     0.7988    0.7509    0.7741      1084
       B-ORG     0.8007    0.8236    0.8120      1400
       B-PER     0.8280    0.8776    0.8520       735
       I-PER     0.8561    0.9385    0.8954       634
       I-ORG     0.8083    0.7409    0.7732      1104
       I-LOC     0.6476    0.6277    0.6375       325

   micro avg     0.8040    0.8006    0.8023      5282
   macro avg     0.7899    0.7932    0.7907      5282
weighted avg     0.8029    0.8006    0.8009      5282

